In [1]:
import pandas as pd

In [2]:
# Load preprocessed data
df = pd.read_csv("financial_news_cleaned.csv")
df

,headline,summary,created_at,ticker,headline_summary,cleaned_text,cleaned_date,historical_price,latest_price,tokens,filtered_tokens,lemmatized
0,How We Missed A Winning Trade On IBM--And How ...,We Won&#39;t Make This Mistake Again\n\n,2024-01-26 15:58:15+00:00,IBM,How We Missed A Winning Trade On IBM--And How ...,How We Missed A Winning Trade On IBMAnd How We...,2024-01-26,187.42,186.90,"['How', 'We', 'Missed', 'A', 'Winning', 'Trade...","['How', 'We', 'Missed', 'A', 'Winning', 'Trade...","['How', 'We', 'Missed', 'A', 'Winning', 'Trade..."
1,Investigating IBM's Standing In IT Services In...,,2024-01-26 16:01:45+00:00,IBM,Investigating IBM's Standing In IT Services In...,Investigating IBMs Standing In IT Services Ind...,2024-01-26,187.42,186.90,"['Investigating', 'IBMs', 'Standing', 'In', 'I...","['Investigating', 'IBMs', 'Standing', 'In', 'I...","['Investigating', 'IBMs', 'Standing', 'In', 'I..."
2,90's Tech Titans Are Outshining 'Magnificent S...,Companies that have strong cash flows will lik...,2024-01-26 12:12:06+00:00,IBM,90's Tech Titans Are Outshining 'Magnificent S...,s Tech Titans Are Outshining Magnificent Seven...,2024-01-26,187.42,186.90,"['s', 'Tech', 'Titans', 'Are', 'Outshining', '...","['Tech', 'Titans', 'Are', 'Outshining', 'Magni...","['Tech', 'Titans', 'Are', 'Outshining', 'Magni..."
3,Why Humana Shares Are Trading Lower By Over 11...,Shares of Humana Inc. (NYSE: HUM) fell sharply...,2024-01-25 19:52:24+00:00,IBM,Why Humana Shares Are Trading Lower By Over 11...,Why Humana Shares Are Trading Lower By Over He...,2024-01-25,190.43,186.90,"['Why', 'Humana', 'Shares', 'Are', 'Trading', ...","['Why', 'Humana', 'Shares', 'Are', 'Trading', ...","['Why', 'Humana', 'Shares', 'Are', 'Trading', ..."
4,"S&P 500 Eyes Sixth Green Session, Tesla Tumble...",U.S. stock market gains due to strong economic...,2024-01-25 19:22:36+00:00,IBM,"S&P 500 Eyes Sixth Green Session, Tesla Tumble...",SP Eyes Sixth Green Session Tesla Tumbles Oil ...,2024-01-25,190.43,186.90,"['SP', 'Eyes', 'Sixth', 'Green', 'Session', 'T...","['SP', 'Eyes', 'Sixth', 'Green', 'Session', 'T...","['SP', 'Eyes', 'Sixth', 'Green', 'Session', 'T..."
...,...,...,...,...,...,...,...,...,...,...,...,...
6246,You Can Have Any Color Tesla Model X To Qualif...,While stealth grey is included in the base pri...,2024-01-18 09:47:05+00:00,TSLA,You Can Have Any Color Tesla Model X To Qualif...,You Can Have Any Color Tesla Model X To Qualif...,2024-01-18,211.88,188.86,"['You', 'Can', 'Have', 'Any', 'Color', 'Tesla'...","['You', 'Can', 'Have', 'Any', 'Color', 'Tesla'...","['You', 'Can', 'Have', 'Any', 'Color', 'Tesla'..."
6247,Tesla Invites Chinese Fans To Decide Cybertruc...,"In China, pickup trucks are classified as ligh...",2024-01-18 08:50:06+00:00,TSLA,Tesla Invites Chinese Fans To Decide Cybertruc...,Tesla Invites Chinese Fans To Decide Cybertruc...,2024-01-18,211.88,188.86,"['Tesla', 'Invites', 'Chinese', 'Fans', 'To', ...","['Tesla', 'Invites', 'Chinese', 'Fans', 'To', ...","['Tesla', 'Invites', 'Chinese', 'Fans', 'To', ..."
6248,Tesla Loses Most-Shorted Crown To This 'Magnif...,Apple and Microsoft have around $18.63 billion...,2024-01-18 07:38:57+00:00,TSLA,Tesla Loses Most-Shorted Crown To This 'Magnif...,Tesla Loses MostShorted Crown To This Magnific...,2024-01-18,211.88,188.86,"['Tesla', 'Loses', 'MostShorted', 'Crown', 'To...","['Tesla', 'Loses', 'MostShorted', 'Crown', 'To...","['Tesla', 'Loses', 'MostShorted', 'Crown', 'To..."
6249,China's Top Auto Makers Huddle In Beijing To P...,Early signs of a slowdown in the Chinese auto ...,2024-01-18 06:54:29+00:00,TSLA,China's Top Auto Makers Huddle In Beijing To P...,Chinas Top Auto Makers Huddle In Beijing To Pl...,2024-01-18,211.88,188.86,"['Chinas', 'Top', 'Auto', 'Makers', 'Huddle', ...","['Chinas', 'Top', 'Auto', 'Makers', 'Huddle', ...","['Chinas', 'Top', 'Auto', 'Makers', 'Huddle', ..."


## Calculation of a composite score / the target variable
<br>
Calculating a composite score from 0 to 1 to indicate how likely an article is to be indicative of stock growth, we integrate the following aspects:
<br>
1. **Impact Score** (based on the presence of keywords related to direct stock impact)
<br>
2. **Forecast Score** (reflecting the frequency of forward-looking or predictive keywords in the text)
<br>
3. **Sentiment Analysis** (including both sentiment score and sentiment label)
<br>
4. **Recency** (prioritizing more recent articles)
<br>
5. **Stock Growth Analysis** (considering the recent growth of the stock mentioned in the article, combined with sentiment)

## Impact analysis

In [3]:
# Notes: The weights are subjective and could be refined based on specific 
# industry knowledge or empirical analysis of past market reactions to similar news.
impact_keywords = {
    "earnings": 0.7, "revenue": 0.6, "profit": 0.6, "loss": 0.7, "EPS": 0.5,
    "guidance": 0.7, "forecast": 0.7, "dividend": 0.5,
    "merger": 0.75, "acquisition": 0.75, "takeover": 0.7,
    "bankruptcy": 0.8, "restructuring": 0.7, "layoff": 0.7,
    "lawsuit": 0.75, "patent": 0.5, "approval": 0.6,
    "IPO": 0.7, "stock split": 0.5, "market share": 0.6,
    "CEO": 0.6, "CFO": 0.5, "resignation": 0.7,
    "expansion": 0.7, "contract": 0.6, "partnership": 0.5,
    "regulation": 0.75, "compliance": 0.5, "trade war": 0.8,
    "interest rate": 0.75, "inflation": 0.7, "economic growth": 0.7,
    "funding": 0.5, "investment": 0.6, "venture capital": 0.5,
    "debt": 0.7, "credit": 0.5, "default": 0.8,
    "product launch": 0.6, "innovation": 0.7, "R&D": 0.6,
    "shareholder": 0.5, "stakeholder": 0.4, "buyout": 0.75,
    "sustainability": 0.5, "ESG": 0.5, "environmental": 0.5,
    "litigation": 0.75, "fraud": 0.8, "scandal": 0.75,
    "data breach": 0.75, "cybersecurity": 0.6, "privacy": 0.5,
    "regulatory approval": 0.7, "antitrust": 0.75, "monopoly": 0.7
}

forecast_keywords = {
    "forecast": 0.7, "projection": 0.6, "outlook": 0.7, "guidance": 0.7,
    "expansion plans": 0.75, "growth strategy": 0.75, "investment plan": 0.6,
    "R&D pipeline": 0.6, "capital expenditure": 0.5,
    "market penetration strategy": 0.6, "innovation roadmap": 0.7,
    "strategic initiatives": 0.6, "long-term goals": 0.5,
    "product roadmap": 0.6, "market expansion": 0.7,
    "diversification strategy": 0.5, "sustainability initiatives": 0.5,
    "merger & acquisition plans": 0.8, "supply chain optimization": 0.5,
    "technological advancements": 0.7
}

def calculate_score(text, keywords):
    score = 0.0
    # Convert text to lowercase for case-insensitive matching
    lower_text = text.lower()  
    for keyword, weight in keywords.items():
        if keyword in lower_text:
            score += weight
    return score

df['impact_score'] = df['filtered_tokens'].apply(lambda x: calculate_score(x, impact_keywords))
df['forecast_score'] = df['filtered_tokens'].apply(lambda x: calculate_score(x, forecast_keywords))
df

,headline,summary,created_at,ticker,headline_summary,cleaned_text,cleaned_date,historical_price,latest_price,tokens,filtered_tokens,lemmatized,impact_score,forecast_score
0,How We Missed A Winning Trade On IBM--And How ...,We Won&#39;t Make This Mistake Again\n\n,2024-01-26 15:58:15+00:00,IBM,How We Missed A Winning Trade On IBM--And How ...,How We Missed A Winning Trade On IBMAnd How We...,2024-01-26,187.42,186.90,"['How', 'We', 'Missed', 'A', 'Winning', 'Trade...","['How', 'We', 'Missed', 'A', 'Winning', 'Trade...","['How', 'We', 'Missed', 'A', 'Winning', 'Trade...",0.0,0.0
1,Investigating IBM's Standing In IT Services In...,,2024-01-26 16:01:45+00:00,IBM,Investigating IBM's Standing In IT Services In...,Investigating IBMs Standing In IT Services Ind...,2024-01-26,187.42,186.90,"['Investigating', 'IBMs', 'Standing', 'In', 'I...","['Investigating', 'IBMs', 'Standing', 'In', 'I...","['Investigating', 'IBMs', 'Standing', 'In', 'I...",0.0,0.0
2,90's Tech Titans Are Outshining 'Magnificent S...,Companies that have strong cash flows will lik...,2024-01-26 12:12:06+00:00,IBM,90's Tech Titans Are Outshining 'Magnificent S...,s Tech Titans Are Outshining Magnificent Seven...,2024-01-26,187.42,186.90,"['s', 'Tech', 'Titans', 'Are', 'Outshining', '...","['Tech', 'Titans', 'Are', 'Outshining', 'Magni...","['Tech', 'Titans', 'Are', 'Outshining', 'Magni...",0.0,0.0
3,Why Humana Shares Are Trading Lower By Over 11...,Shares of Humana Inc. (NYSE: HUM) fell sharply...,2024-01-25 19:52:24+00:00,IBM,Why Humana Shares Are Trading Lower By Over 11...,Why Humana Shares Are Trading Lower By Over He...,2024-01-25,190.43,186.90,"['Why', 'Humana', 'Shares', 'Are', 'Trading', ...","['Why', 'Humana', 'Shares', 'Are', 'Trading', ...","['Why', 'Humana', 'Shares', 'Are', 'Trading', ...",1.4,0.7
4,"S&P 500 Eyes Sixth Green Session, Tesla Tumble...",U.S. stock market gains due to strong economic...,2024-01-25 19:22:36+00:00,IBM,"S&P 500 Eyes Sixth Green Session, Tesla Tumble...",SP Eyes Sixth Green Session Tesla Tumbles Oil ...,2024-01-25,190.43,186.90,"['SP', 'Eyes', 'Sixth', 'Green', 'Session', 'T...","['SP', 'Eyes', 'Sixth', 'Green', 'Session', 'T...","['SP', 'Eyes', 'Sixth', 'Green', 'Session', 'T...",0.7,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6246,You Can Have Any Color Tesla Model X To Qualif...,While stealth grey is included in the base pri...,2024-01-18 09:47:05+00:00,TSLA,You Can Have Any Color Tesla Model X To Qualif...,You Can Have Any Color Tesla Model X To Qualif...,2024-01-18,211.88,188.86,"['You', 'Can', 'Have', 'Any', 'Color', 'Tesla'...","['You', 'Can', 'Have', 'Any', 'Color', 'Tesla'...","['You', 'Can', 'Have', 'Any', 'Color', 'Tesla'...",0.5,0.0
6247,Tesla Invites Chinese Fans To Decide Cybertruc...,"In China, pickup trucks are classified as ligh...",2024-01-18 08:50:06+00:00,TSLA,Tesla Invites Chinese Fans To Decide Cybertruc...,Tesla Invites Chinese Fans To Decide Cybertruc...,2024-01-18,211.88,188.86,"['Tesla', 'Invites', 'Chinese', 'Fans', 'To', ...","['Tesla', 'Invites', 'Chinese', 'Fans', 'To', ...","['Tesla', 'Invites', 'Chinese', 'Fans', 'To', ...",0.0,0.0
6248,Tesla Loses Most-Shorted Crown To This 'Magnif...,Apple and Microsoft have around $18.63 billion...,2024-01-18 07:38:57+00:00,TSLA,Tesla Loses Most-Shorted Crown To This 'Magnif...,Tesla Loses MostShorted Crown To This Magnific...,2024-01-18,211.88,188.86,"['Tesla', 'Loses', 'MostShorted', 'Crown', 'To...","['Tesla', 'Loses', 'MostShorted', 'Crown', 'To...","['Tesla', 'Loses', 'MostShorted', 'Crown', 'To...",0.0,0.0
6249,China's Top Auto Makers Huddle In Beijing To P...,Early signs of a slowdown in the Chinese auto ...,2024-01-18 06:54:29+00:00,TSLA,China's Top Auto Makers Huddle In Beijing To P...,Chinas Top Auto Makers Huddle In Beijing To Pl...,2024-01-18,211.88,188.86,"['Chinas', 'Top', 'Auto', 'Makers', 'Huddle', ...","['Chinas', 'Top', 'Auto', 'Makers', 'Huddle', ...","['Chinas', 'Top', 'Auto', 'Makers', 'Huddle', ...",0.0,0.0


## Sentiment analysis 
<br>
First apply FinBERT to dataset

In [4]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline
import numpy as np

# Load pre-trained model and tokenizer
model_name = "yiyanghkust/finbert-tone"  # Example FinBERT model
model = BertForSequenceClassification.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

# Create a sentiment analysis pipeline
finbert = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# Function to apply FinBERT to a batch of texts
def finbert_helper(texts):
    results = finbert(texts)
    labels = [result['label'] for result in results]
    scores = [result['score'] for result in results]
    return labels, scores

# Batch processing with FinBERT
def apply_finbert_to_batch(dataframe, column_name, batch_size=20):
    num_batches = int(np.ceil(len(dataframe) / batch_size))
    labels, scores = [], []

    for i in range(num_batches):
        batch_texts = dataframe[column_name].iloc[i * batch_size:(i + 1) * batch_size].tolist()
        batch_labels, batch_scores = finbert_helper(batch_texts)
        labels.extend(batch_labels)
        scores.extend(batch_scores)

    return labels, scores

df['sentiment_label'], df['sentiment_score'] = apply_finbert_to_batch(df, 'headline_summary', batch_size=20)

In [5]:
# Adjust Sentiment Score - we want a lower composite score for articles with a negative sentiment
def adjust_sentiment_score(sentiment_label, sentiment_score):
    label_to_numeric = {"Positive": 1, "Neutral": 0.5, "Negative": 0}
    
    adjusted_score = sentiment_score * label_to_numeric[sentiment_label]
    return adjusted_score

df['adjusted_sentiment_score'] = df.apply(lambda x: adjust_sentiment_score(x['sentiment_label'], x['sentiment_score']), axis=1)

In [6]:
'''
Testing for sentiment analysis
'''
import random

test_num = random.randint(1, 6000)
print(df.iloc[test_num].headline)
print(df.iloc[test_num].sentiment_label)
print(df.iloc[test_num].sentiment_score)
print(df.iloc[test_num].adjusted_sentiment_score)

General Electric Analyst Downgrades Stock After 2023 Gains
Positive
0.9999866485595703
0.9999866485595703


## Calculating Recency Score
<br>
Assigns higher scores to more recent articles, using a exponential decay based on the date of 
the article relative to the current date.

In [10]:
from datetime import datetime
import math

def calculate_recency_score(publication_date, current_date=datetime.now(), half_life=30):
    # Convert publication_date to datetime if it's a string
    if isinstance(publication_date, str):
        publication_date = datetime.strptime(publication_date, "%Y-%m-%d")

    # Calculate the difference in days between the current date and the publication date
    days_diff = (current_date - publication_date).days
    
    # Calculate the exponential decay
    # The formula is: e^(-ln(2) * days_diff / half_life)
    decay = math.exp(-math.log(2) * days_diff / half_life)
    
    return decay

df['recency_score'] = df['cleaned_date'].apply(calculate_recency_score)

## Stock Growth Analysis

In [32]:
def calculate_stock_growth(historical_price, current_price):
    if historical_price and current_price:
        growth = ((current_price - historical_price) / historical_price) * 100
        return growth
    else:
        return None
    
df['stock_growth'] = df.apply(lambda x: calculate_stock_growth(x['historical_price'], x['latest_price']), axis=1)

# Converts stock growth percentage to a score between 0 and 1.
# Growth of 6% or above is considered maximum value (score of 1).
def growth_to_score(growth_percentage, max_growth=6):
    if growth_percentage >= max_growth:
        return 1
    elif growth_percentage <= 0:
        return 0
    else:
        # Linear scaling for growth percentage between 0% and max_growth%
        return growth_percentage / max_growth

df['growth_score'] = df['stock_growth'].apply(lambda x: growth_to_score(x))

In [34]:
def calculate_composite_score(row, max_impact_score, max_forecast_score, weight_impact=0.25, weight_forecast=0.15, weight_sentiment=0.20, weight_recency=0.15, weight_growth=0.15):
    normalised_impact_score = row['impact_score'] / max_impact_score
    normalised_forecast_score = row['forecast_score'] / max_forecast_score
    # Already normalised
    sentiment_score = row['adjusted_sentiment_score']  
    recency_score = row['recency_score']  
    growth_score = row['growth_score']  
    
    composite_score = (normalised_impact_score * weight_impact) + \
                      (normalised_forecast_score * weight_forecast) + \
                      (sentiment_score * weight_sentiment) + \
                      (recency_score * weight_recency) + \
                      (growth_score * weight_growth)
    
    return round(composite_score, 4)

# Calculate max_impact_score and max_forecast_score for normalisation
max_impact_score = df['impact_score'].max()
max_forecast_score = df['forecast_score'].max()

# Apply the calculate_composite_score function to each row
df['score'] = df.apply(lambda row: calculate_composite_score(row, max_impact_score, max_forecast_score), axis=1)

print(df['score'].max())
print(df['score'].min())

0.7251
0.0


In [35]:
# Save dataset after feature-engineered
df.to_csv('financial_news_wfeatures.csv', index=False)